# Combining SLAM with Gaze Tracing

To start, we need to import the necessary libraries

In [1]:
import pathlib
import time
import os

import trimesh
import pandas as pd
import pyorbslam
import cv2
import numpy as np
import imutils

# And let's get the path to the GIT directory and other import directories
GIT_ROOT = pathlib.Path(os.path.abspath('')).parent
SETTINGS_DIR = GIT_ROOT / 'settings'
MONITOR_PLY = GIT_ROOT / 'notebooks' / 'data' / 'monitor.ply'
OUTPUTS_DIR = GIT_ROOT / 'notebooks' / 'outputs'
N = 1000

# Loading ORBSLAM3

Creating a SLAM object with the right settings file

In [2]:
slam = pyorbslam.MonoSLAM(SETTINGS_DIR / 'Tobii.yaml')
drawer = pyorbslam.TrajectoryDrawer(port=9000)


ORB-SLAM3 Copyright (C) 2017-2020 Carlos Campos, Richard Elvira, Juan J. Gómez, José M.M. Montiel and Juan D. Tardós, University of Zaragoza.
ORB-SLAM2 Copyright (C) 2014-2016 Raúl Mur-Artal, José M.M. Montiel and Juan D. Tardós, University of Zaragoza.
This program comes with ABSOLUTELY NO WARRANTY;
This is free software, and you are welcome to redistribute it
under certain conditions. See LICENSE.txt.

Input sensor was set to: Monocular
Loading settings from /home/eduardo/GitHub/pyorbslam/settings/Tobii.yaml
	-Loaded camera 1
	-Loaded image info
	-Loaded ORB settings
	-Loaded viewer settings
	-Loaded Atlas settings
	-Loaded misc parameters
----------------------------------
SLAM settings: 
	-Camera 1 parameters (Pinhole): [ 910.596 910.208 958.434 511.661 ]
	-Camera 1 distortion parameters: [  -0.0559193 0.0797811 -0.048538 -0.000144261 ]
	-Original image size: [ 1920 , 1080 ]
	-Current image size: [ 600 , 350 ]
	-Camera 1 parameters after resize: [  284.561 294.975 299.511 165.816 

Camera1.k3 optional parameter does not exist...
Viewer.imageViewScale optional parameter does not exist...
System.LoadAtlasFromFile optional parameter does not exist...
System.SaveAtlasToFile optional parameter does not exist...
System.thFarPoints optional parameter does not exist...


2023-07-18 02:18:00 [DEBUG] pyorbslam: <HttpServer>: Running at localhost:9000
2023-07-18 02:18:00 [DEBUG] pyorbslam: <pyorbslam.trajectory_drawer.td_app.threaded_zmq_poller.ThreadedZmqPoller object at 0x7f0e2dd1e950>: Polling thread running
2023-07-18 02:18:01 [DEBUG] pyorbslam: <pyorbslam.trajectory_drawer.publisher.Publisher object at 0x7f0e0e5a64d0>: Successful creating SUB/PUB connection!


# Processing Video and Drawing the Gaze

In [8]:
# Load the video
video_path = GIT_ROOT/'test'/'data'/'scenevideo.mp4'
assert video_path.exists()
cap = cv2.VideoCapture(str(video_path))

# Resetting the SLAM every time before running on video
slam.reset()
drawer.reset()

# Necessary things to track the progress of SLAM
timestamp = 0
fps = 1/24
i = 0

# for i in range(N):
for i in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):

    # Loading the frame
    tic = time.time()
    ret, frame = cap.read()

    # Process the frame
    state = slam.process(frame, timestamp)

    # Only if things work out should we visualize the state
    if state == pyorbslam.State.OK:
        pose = slam.get_pose_to_target()
        drawer.plot_trajectory(pose)
        drawer.plot_pointcloud('pc', slam.get_point_cloud())
        pyorbslam.tools.record_slam_data(i, slam, OUTPUTS_DIR / 'tobii')

    # Update
    i += 1
    timestamp += fps

    # Show
    drawer.plot_image(imutils.resize(frame, width=200))

TrackMonocular Start
LM: Map reset recieved
LM: Map reset, waiting...
LM: Reseting Atlas in Local Mapping...
LM: End reseting Local Mapping...
LM: Reset free the mutex
LM: Map reset, Done!!!
Loop closer reset requested...
Creation of new map with id: 4
Creation of new map with last KF id: 0
TrackMonocular 1
TrackMonocular 2
TrackMonocular End
TrackMonocular Start
TrackMonocular 1
TrackMonocular 2
TrackMonocular End
TrackMonocular Start
TrackMonocular 1
TrackMonocular 2
TrackMonocular End
TrackMonocular Start
TrackMonocular 1
TrackMonocular 2
TrackMonocular End
TrackMonocular Start
TrackMonocular 1
TrackMonocular 2
TrackMonocular End
TrackMonocular Start
TrackMonocular 1
TrackMonocular 2
TrackMonocular End
TrackMonocular Start
TrackMonocular 1
TrackMonocular 2
TrackMonocular End
TrackMonocular Start
TrackMonocular 1
TrackMonocular 2
TrackMonocular End
TrackMonocular Start
TrackMonocular 1
TrackMonocular 2
TrackMonocular End
TrackMonocular Start
TrackMonocular 1
TrackMonocular 2
TrackMon

# Let's visualize the results

In [5]:
# Resetting the SLAM every time before running on video
drawer.reset()

# 3D object information
correct_rt = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, -1, 0, 0],
    [0, 0, 0, 1]
])
t = np.array([0.9, -0.15, 1.5])
r = np.array([-0.1, 0.5, 0.18])
R, _ = cv2.Rodrigues(r.astype(np.float32))
rt = np.eye(4)
rt[:3, :3] = R
rt[:3, 3] = t

# Load 3D model
monitor: trimesh.Trimesh = trimesh.load_mesh(str(MONITOR_PLY))
monitor = monitor.apply_scale(0.03)
monitor = monitor.apply_transform(rt)
monitor = monitor.apply_transform(correct_rt)

# Configure monitor
drawer.add_mesh('monitor', monitor, color=(0,1,0,1))

# Create the gaze line
line = np.array([
    [0,0,0],
    [0,0,1]
])

# Load the data now
for i in range(N):
    # load
    data = pyorbslam.tools.load_slam_data(i, OUTPUTS_DIR / 'tobii')

    # Visualize
    if data:

        # Compute gaze vector
        gaze_vector = pyorbslam.tools.apply_rt_to_pts(line, data['pose'])

        # Draw gaze on the image
        img = data['image']
        h, w = img.shape[:2]
        fix = (int(w/2),int(h/2))
        draw_frame = cv2.circle(img, fix, 10, (0,0,255), 3)

        drawer.plot_path(gaze_vector)
        drawer.plot_image(draw_frame)
        drawer.plot_trajectory(data['pose'])
        drawer.plot_pointcloud('pc', data['point cloud'])

    time.sleep(1/60)
